In [1]:
import h5py
import pandas as pd
import json
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ssp
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from collections import Counter
import pickle
from sklearn.utils import shuffle
from os.path import join as pjoin
from tqdm.auto import tqdm
import os
import joblib
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import importlib 
from sklearn.preprocessing import normalize as sk_normalize
import pytorch_lightning as pl
from os.path import join as pjoin
from scipy import sparse

In [2]:
data_dir = "/data/project/rw/recsys2022/vali_2105"

In [3]:
def build_dataset(sessions, purchases=None):
    item_feats = pd.read_csv(pjoin(data_dir,'item_features.csv'), dtype=str)
    item_feats['feat_id'] = item_feats['feature_category_id'] + ':' + item_feats['feature_value_id']
    d = item_feats.groupby("item_id")['feat_id'].apply(list).reset_index()
    item_to_feats = {x:y for (x, y) in zip(d.item_id, d.feat_id)}

    def items_to_feats(item_ids, item_to_feats):
        ret = []
        for x in item_ids:
            ret.extend(item_to_feats[x])
        return ret

    sessions['kp'] = [(x,y) for (x,y) in zip(sessions.date, sessions.item_id)]
    t = sessions.groupby('session_id').kp.apply(lambda x: [y for y in sorted(x)]).reset_index()
    q = sessions.groupby('session_id')['date'].apply(lambda x: list(sorted(x))[0])
    v = sessions.groupby('session_id')['date'].apply(lambda x: list(sorted(x))[-1])
    q = pd.merge(q, v, on='session_id')
    print(t.shape, q.shape,v.shape)
    ret = pd.merge(pd.DataFrame(t), pd.DataFrame(q), on='session_id')
    if purchases is not None:
        dataset = pd.merge(ret, purchases, on='session_id')
        dataset.columns = ['session_id', 'item_ids', 'date_session_begin', 'date_session_end', 'target_item', 'date_purchase']
        dataset['target_feat'] = dataset.target_item.apply(lambda x: item_to_feats[x])
    else:
        dataset = ret
        dataset.columns = ['session_id', 'item_ids', 'date_session_begin', 'date_session_end']

    dataset['feats'] = dataset.item_ids.apply(lambda x: items_to_feats([y[1] for y in x], item_to_feats))
    dataset['ym'] = dataset.date_session_end.apply(lambda x:x[:7])
    dataset['year'] = dataset.date_session_end.apply(lambda x:x[:4])
    dataset['month'] = dataset.date_session_end.apply(lambda x:x[5:7])
    dataset['dow'] =  pd.to_datetime(dataset.date_session_end).dt.dayofweek.apply(lambda x: str(x))
    dataset.sort_values(by='date_session_end', inplace=True)
    return dataset

In [4]:
train_sessions = pd.read_csv(pjoin(data_dir, 'train_sessions.csv'), dtype=str)
train_purchases = pd.read_csv(pjoin(data_dir, 'train_purchases.csv'), dtype=str)

In [5]:
train_sessions = train_sessions[train_sessions.date >= '2021-02-01 00:00:00.000']

In [6]:
val_sessions = pd.read_csv(pjoin(data_dir, 'val_sessions.csv'), dtype=str)
val_purchases = pd.read_csv(pjoin(data_dir, 'val_purchases.csv'), dtype=str)

In [7]:
te_sessions = pd.read_csv(pjoin(data_dir, 'te_sessions.csv'), dtype=str)
te_purchases = pd.read_csv(pjoin(data_dir, 'te_purchases.csv'), dtype=str)

In [9]:
_tr_csv = build_dataset(train_sessions, train_purchases)

(178366, 2) (178366, 2) (178366,)


In [10]:
val_csv = build_dataset(val_sessions, val_purchases)

(40808, 2) (40808, 2) (40808,)


In [11]:
te_csv = build_dataset(te_sessions, te_purchases)

(40810, 2) (40810, 2) (40810,)


In [28]:
def flatten(y):
    x = []
    for k in y:
        x.extend(k)
    return x

In [34]:
items = (set(flatten(_tr_csv.item_ids.apply(lambda y: [x[1] for x in y]).tolist())) | 
         set(_tr_csv.target_item.tolist()))

In [54]:
tptp = (val_csv.item_ids.apply(lambda y: len([x[1] for x in y if x[1] not in items])))

In [68]:
(1 / (val_csv.item_ids.apply(lambda y: len(y))[tptp >= 1] / val_csv.item_ids.apply(lambda y: len([x[1] for x in y if x[1] not in items]))[tptp >= 1])).mean()

0.42160078280419716

In [ ]:
tr2 = _tr_csv.copy()

In [ ]:
tr2['target_item'] = tr2['item_ids'].apply(lambda x:x[-1][1])

In [ ]:
tr2['item_ids'] = tr2['item_ids'].apply(lambda x:x[0:-1])

In [ ]:
tr2 = tr2[tr2.item_ids.apply(len) > 0]

In [ ]:
tr_csv = pd.concat([tr2.reset_index(drop=True), _tr_csv.reset_index(drop=True)], axis=0)

In [ ]:
tr_csv = tr_csv.sort_values(by='date_session_end').reset_index(drop=True)

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]
item_feats = pd.read_csv(pjoin(data_dir, 'item_features.csv'), dtype=str)
item_feats['feat_id'] = item_feats['feature_category_id'] + ':' + item_feats['feature_value_id']
d = item_feats.groupby("item_id")['feat_id'].apply(list).reset_index()
item_to_feats = {x:y for (x, y) in zip(d.item_id, d.feat_id)}

In [ ]:
target_cands = set(pd.read_csv(pjoin(data_dir, "candidate_items.csv"), dtype=str).item_id.tolist())

In [ ]:
# tr_csv = _tr_csv.reset_index(drop=True)
val_csv = val_csv.reset_index(drop=True)
te_csv = te_csv.reset_index(drop=True)

In [ ]:
tr_dur = (pd.to_datetime(tr_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(tr_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds
val_dur = (pd.to_datetime(val_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(val_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds
te_dur = (pd.to_datetime(te_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(te_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds

In [ ]:
tr_csv['item_ids'] = tr_csv.item_ids.apply(lambda x : [y[1] for y in x])
val_csv['item_ids'] = val_csv.item_ids.apply(lambda x : [y[1] for y in x])
te_csv['item_ids'] = te_csv.item_ids.apply(lambda x : [y[1] for y in x])

In [ ]:
tr_l = tr_csv.item_ids.apply(len)
val_l = val_csv.item_ids.apply(len)
te_l = te_csv.item_ids.apply(len)

In [ ]:
v_pop_all = Counter(flatten(tr_csv.item_ids.tolist()))
p_pop_all = Counter(tr_csv.target_item.tolist())

In [24]:
tr_v_pop_all = tr_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
tr_p_pop_all = tr_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

val_v_pop_all = val_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
val_p_pop_all = val_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

te_v_pop_all = te_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
te_p_pop_all = te_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

In [25]:
v_pop_3m = Counter(flatten(tr_csv[tr_csv.ym >= "2021-04"].item_ids.tolist()))
p_pop_3m = Counter(tr_csv[tr_csv.ym >= "2021-04"].target_item.tolist())

In [26]:
tr_v_pop_3m = tr_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
tr_p_pop_3m = tr_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

val_v_pop_3m = val_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
val_p_pop_3m = val_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

te_v_pop_3m = te_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
te_p_pop_3m = te_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

In [27]:
tr_s_len = tr_csv.item_ids.apply(len)
val_s_len = val_csv.item_ids.apply(len)
te_s_len = te_csv.item_ids.apply(len)

In [28]:
tr_tmp = pd.concat([
    np.log(1 + tr_s_len),
    np.log(1 + tr_dur),
    np.log(1 + tr_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + tr_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + tr_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + tr_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + tr_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + tr_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + tr_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + tr_p_pop_3m.apply(lambda x: np.max(x))),
    
], axis=1)

val_tmp = pd.concat([
   np.log(1 + val_s_len),
    np.log(1 + val_dur),
    np.log(1 + val_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + val_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + val_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + val_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + val_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + val_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + val_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + val_p_pop_3m.apply(lambda x: np.max(x))),
], axis=1)

te_tmp = pd.concat([
   np.log(1 + te_s_len),
    np.log(1 + te_dur),
    np.log(1 + te_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + te_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + te_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + te_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + te_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + te_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + te_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + te_p_pop_3m.apply(lambda x: np.max(x))),
], axis=1)

tr_scalar = tr_tmp.to_numpy()
val_scalar = val_tmp.to_numpy()
te_scalar = te_tmp.to_numpy()

In [29]:
def iddf(x):
    return x

In [30]:
tr_csv['tt'] = tr_csv.date_session_end.apply(lambda x: x[:9]) + ':' + tr_csv.item_ids.apply(lambda x: ':'.join(sorted(x[-5:])))

In [31]:
kw = tr_csv.groupby('tt').target_item.apply(list).reset_index()

In [32]:
mask = {x: y for (x, y) in zip(kw.tt, kw.target_item)}

In [33]:
input_mask_all = tr_csv.item_ids# + tr_csv.tt.apply(lambda x: mask[x])

In [34]:
ret = []
for x, y in  zip(input_mask_all, tr_csv.target_item):
    r = [k for k in x if k != y]
    ret.append(r)

In [35]:
input_mask_all = ret

In [36]:
lm_item = sorted(list(set(flatten(tr_csv.item_ids.apply(lambda x: x))) | set(tr_csv.target_item.unique())), key=lambda x: int(x))
# lm_item = set(item_to_feats.keys())
id2idx = {x:i for i,x in enumerate(lm_item)}
item_CV = CountVectorizer(analyzer =iddf, vocabulary=id2idx)

In [37]:
tr_mask_all = item_CV.transform(input_mask_all)

In [38]:
AA = tr_mask_all.toarray().sum(-1).ravel()

In [39]:
BB = item_CV.transform(tr_csv.item_ids).toarray().sum(-1).ravel()

In [40]:
(AA - BB).mean()

-0.1095497638159416

In [41]:
tr_targets = tr_csv.target_item.apply(lambda x: id2idx[x]).tolist()
val_targets = val_csv.target_item.apply(lambda x: id2idx.get(x, 29999)).tolist()
te_targets = te_csv.target_item.apply(lambda x: id2idx.get(x, 29999)).tolist()
(np.array(val_targets) == 29999).sum(), (np.array(te_targets) == 29999).sum()

(5059, 4932)

In [42]:
item_all = item_CV.transform(tr_csv.item_ids.apply(lambda x: x))
last_20 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-20:]))
last_5 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-5:]))
last_2 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-2:]))
last_1 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-1:]))

val_all = item_CV.transform(val_csv.item_ids.apply(lambda x: x))
val_last_20 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-20:]))
val_last5 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-5:]))
val_last2 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-2:]))
val_last1 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-1:]))

te_all = item_CV.transform(te_csv.item_ids.apply(lambda x: x))
te_last_20 = item_CV.transform(te_csv.item_ids.apply(lambda x: x[-20:]))
te_last5 = item_CV.transform(te_csv.item_ids.apply(lambda x: x[-5:]))
te_last2 = item_CV.transform(te_csv.item_ids.apply(lambda x: x[-2:]))
te_last1 = item_CV.transform(te_csv.item_ids.apply(lambda x: x[-1:]))

In [43]:
tr_last_5_feats = tr_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
tr_last_3_feats = tr_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
tr_last_1_feats = tr_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))

val_last_5_feats = val_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
val_last_3_feats = val_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
val_last_1_feats = val_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))

te_last_5_feats = te_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
te_last_3_feats = te_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
te_last_1_feats = te_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))



In [44]:
year_map = {y: x for (x, y) in enumerate(['2020', '2021'])}
month_map = {y: x for (x, y) in enumerate(["%02d" % x for x in range(1, 13)])}
hour_map = {y: x for (x, y) in enumerate(["%d" % x for x in range(0, 24)])}
dow_map = {y: x for (x, y) in enumerate(tr_csv.dow.unique())}
tr_csv['dow'] = pd.to_datetime(tr_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
tr_csv['hour'] = pd.to_datetime(tr_csv.date_session_end).dt.hour.apply(lambda x: str(x))

val_csv['dow'] = pd.to_datetime(val_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
val_csv['hour'] = pd.to_datetime(val_csv.date_session_end).dt.hour.apply(lambda x: str(x))

te_csv['dow'] = pd.to_datetime(te_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
te_csv['hour'] = pd.to_datetime(te_csv.date_session_end).dt.hour.apply(lambda x: str(x))

In [45]:
all_feats = flatten([item_to_feats[x] for x in lm_item])

In [46]:
all_feats = set(all_feats)

In [47]:
all_feats = set(flatten(item_to_feats.values()))

In [48]:
i2fidx = item_to_feats.keys()
feat_CV = CountVectorizer(analyzer=iddf, 
                          vocabulary=all_feats)

In [49]:
lm_feats = normalize(feat_CV.transform([item_to_feats[x] for x in lm_item]), norm='l1')

In [50]:
tr_l5_feats = normalize(feat_CV.transform(tr_last_5_feats), norm='l1').toarray()
tr_l3_feats = normalize(feat_CV.transform(tr_last_3_feats), norm='l1').toarray()
tr_l1_feats = normalize(feat_CV.transform(tr_last_1_feats), norm='l1').toarray()

val_l5_feats = normalize(feat_CV.transform(val_last_5_feats), norm='l1').toarray()
val_l3_feats = normalize(feat_CV.transform(val_last_3_feats), norm='l1').toarray()
val_l1_feats = normalize(feat_CV.transform(val_last_1_feats), norm='l1').toarray()

te_l5_feats = normalize(feat_CV.transform(te_last_5_feats), norm='l1').toarray()
te_l3_feats = normalize(feat_CV.transform(te_last_3_feats), norm='l1').toarray()
te_l1_feats = normalize(feat_CV.transform(te_last_1_feats), norm='l1').toarray()



In [51]:
EMBED_DIM = 16
tr_cats = {
#     "year": (tr_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (tr_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (tr_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (tr_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),
}

val_cats = {
#     "year": (val_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (val_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (val_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (val_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),    
}

te_cats = {
#     "year": (te_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (te_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (te_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (te_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),    
}

In [52]:
tr_mats = {
    "item_bow": item_all, 
    "item_last_20": last_20,
    "item_last_5": last_5,
    "item_last_2": last_2,
    "item_last_1": last_1,
#     'item_all_feats': tr_feats,    
    'item_last_5_feats': tr_l5_feats,
    "item_last_3_feats" : tr_l3_feats,
    "item_last_1_feats" : tr_l1_feats,
#     "last_hits": tr_vm_hit,
}

val_mats  = {
    "item_bow": val_all, 
    "item_last_20": val_last_20,
    "item_last_5": val_last5,
    "item_last_2": val_last2,
    "item_last_1": val_last1,
#     'item_all_feats': val_feats,    
    'item_last_5_feats': val_l5_feats,
    "item_last_3_feats" : val_l3_feats,
    "item_last_1_feats" : val_l1_feats,
#     "item_last_5_feats": val_last_5_feats,
#     "last_hits": val_vm_hit,
}

te_mats  = {
    "item_bow": te_all, 
    "item_last_20": te_last_20,
    "item_last_5": te_last5,
    "item_last_2": te_last2,
    "item_last_1": te_last1,
#     'item_all_feats': te_feats,    
    'item_last_5_feats': te_l5_feats,
    "item_last_3_feats" : te_l3_feats,
    "item_last_1_feats" : te_l1_feats,
#     "item_last_5_feats": te_last_5_feats,
#     "last_hits": te_vm_hit,
}

In [53]:
import mlps
importlib.reload(mlps)

<module 'mlps' from '/data/project/rw/recsys-challenge-2022-2105/mlp/mlps.py'>

In [54]:
val_mask_all = item_CV.transform(val_csv.item_ids)
te_mask_all = item_CV.transform(te_csv.item_ids)

In [55]:
tr_ds = mlps.dictDataset(tr_mask_all, tr_mats, tr_cats, tr_scalar, tr_targets)
val_ds = mlps.dictDataset(val_mask_all, val_mats, val_cats, val_scalar, val_targets)
te_ds = mlps.dictDataset(te_mask_all, te_mats, te_cats, te_scalar, te_targets)

In [71]:
!rm -rf ./procseed_data_with_aug
!mkdir ./procseed_data_with_aug

In [56]:
fname ='procseed_data_with_aug'

In [57]:
joblib.dump(lm_feats, '%s/lm_feats' % fname)

['procseed_data_with_aug/lm_feats']

In [58]:
joblib.dump(val_ds, "%s/val_ds" % fname)
joblib.dump(tr_ds, '%s/tr_ds' % fname)
joblib.dump(te_ds, '%s/te_ds' % fname)
joblib.dump(te_csv, "%s/te_csv" % fname)
joblib.dump(val_csv, "%s/val_csv" % fname)
joblib.dump(tr_csv, '%s/tr_csv' % fname)
joblib.dump(id2idx, '%s/id2idx' % fname )
joblib.dump((item_CV, feat_CV), "%s/cv" % fname)

['procseed_data_with_aug/cv']